In [ ]:
!pip install mediapipe opencv-python

In [ ]:
 import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [ ]:
from google.colab import drive # 국방부 해커톤/input에 있는 영상 사본 만들고 사본을 옮겨서 이름 바꾸기 / 원본 안옮기게 주의!
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.chdir('/content/gdrive/MyDrive/국방부 해커톤/input/팔굽혀펴기')

# **Finding XYZ**

In [ ]:
def find_xyz(ind_list, landmark):
  a = landmark[ind_list[0]]
  b = landmark[ind_list[1]]
  c = landmark[ind_list[2]]

  first = [a.x,a.y,a.z]
  mid = [b.x,b.y,b.z]
  end = [c.x,c.y,c.z]
  return first, mid, end

# **Calculating Angle**

In [ ]:
def calculate_angle3D(a,b,c,direction):
  """
  calculate_angle3D is divided by left and right side because this function uses external product
  input : a,b,c -> landmarks with shape [x,y,z,visibility]
          direction -> int -1 or 1
                      -1 means Video(photo) for a person's left side and 1 means Video(photo) for a person's right side
  output : angle between vector ba and bc with range 0~360
  """
  # external product's z value
  external_z = (b[0]-a[0])*(b[1]-c[1]) - (b[1]-a[1])*(b[0]-c[0])

  a = np.array(a) #first
  b = np.array(b) #mid
  c = np.array(c) #end

  ba = b-a
  bc = b-c
  dot_result = np.dot(ba, bc)


  ba_size = np.linalg.norm(ba)
  bc_size = np.linalg.norm(bc)
  radi = np.arccos(dot_result / (ba_size*bc_size))
  angle = np.abs(radi*180.0/np.pi)

  # left side
  if external_z * direction > 0:
    angle = 360 - angle

  return angle


In [ ]:
def calculate_angle2D(a,b,c,direction):
  """
  calculate_angle2D is divided by left and right side because this function uses external product
  input : a,b,c -> landmarks with shape [x,y,z,visibility]
          direction -> int -1 or 1
                      -1 means Video(photo) for a person's left side and 1 means Video(photo) for a person's right side
  output : angle between vector ba and bc with range 0~360
  """
  # external product's z value
  external_z = (b[0]-a[0])*(b[1]-c[1]) - (b[1]-a[1])*(b[0]-c[0])

  a = np.array(a[:2]) #first
  b = np.array(b[:2]) #mid
  c = np.array(c[:2]) #end

  ba = b-a
  bc = b-c
  dot_result = np.dot(ba, bc)


  ba_size = np.linalg.norm(ba)
  bc_size = np.linalg.norm(bc)
  radi = np.arccos(dot_result / (ba_size*bc_size))
  angle = np.abs(radi*180.0/np.pi)

  if external_z * direction > 0:
    angle = 360 - angle

  return angle

#**Checking Conditions for Pushups**

##2D angle

In [ ]:
def IsOutlier_pushups(angle_list, joint):
  """
  한 프레임 차이인데 각도차이가 너무 많이 나면 그 값을 무시함.
  joint는 0, 1, 2 값을 가지며 각각 elbow, hip, knee를 나타냄
  """
  if len(angle_list) < 4:
    return False
  
  th = [40, 40, 30]

  # 가장 최근 값 3개와 새로운 값이 차이가 너무 크면 outlier
  if abs((angle_list[-2]+angle_list[-3]+angle_list[-4])/3 - angle_list[-1]) > th[joint]:
      return True
  return False
  

In [ ]:
video_path = "/content/gdrive/MyDrive/국방부 해커톤/input/팔굽혀펴기/4.mp4"
cap = cv2.VideoCapture(video_path)
elbow_state = 'up'
Is_start = False

# 팔굽혀펴기 1회 측정 시간
start = 0
end = 0

#우측 관절값
joint_indx = {'right_elbow':[16,14,12],'right_hip':[12,24,26],'right_knee':[24,26,28]} 

"""
temp은 팔굽혀펴기 운동을 한번 할떄마다(내려갔다 올라올 때마다) 해당 관절 부분의 각도를 최신화함. 이전에 했던 팔굽혀펴기의 각도값들은 다 초기화함.
      -> 앱 상에서 관절 각도값을 저장하는 데에 무리가 가지않게 list를 초기화시켜줌.
"""
temp = {'right_elbow':[],'right_hip':[],'right_knee':[]}
pushups = []

"""
pushups는 list(list)형식이다. 안쪽 list는 length가 5이며 각각 Is_elbow_up, Is_elbow_down, hip_condition, knee_condition, Is_speed_good을 의미한다.
Is_elbow_up -> int 0 or 1 -> 팔꿈치를 완전히 펴면 1, 완전히 펴지 않으면 0
Is_elbow_down -> int 0 or 1 -> 팔꿈치를 완전히 굽히면 1, 완전히 굽히지 않으면 0
hip_condition -> int 0 or 1 or 2 -> 골반이 정상이면 0, 너무 내려가면 1, 너무 올라가면 2
knee_condition -> int 1 or 0 -> 무릎 각도가 정상범위면 1, 너무 내려가면 0
Is_speed_good -> int 1 or 0 -> 팔굽혀펴기 속도가 적당하면(1회당 1초 이상) 1, 너무 빠르면(1회당 1초 미만) 0 
예를 들어, pushup[0]이 [1,0,1,1,1]이면 첫번째 푸쉬업 동작이 올라올 때 팔꿈치를 완전히 폈고, 내려갈때 완전히 팔을 굽히지는 않았으며, 골반은 과도하게 내려갔고,
무릎각도는 정상범위이며 속도가 너무 빠르지않다는 의미이다.
"""

if cap.isOpened():
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

  res=(int(width), int(height))
  fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
  out = cv2.VideoWriter('4test.mp4', fourcc, 20.0, res)

  frame = None
  while True:
    try:
      ret, frame = cap.read()
    except cv2.error:
      continue
    if not ret:
      break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    if results.pose_landmarks:
      mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
      landmark = results.pose_landmarks.landmark
      for key,val in joint_indx.items():
        first,mid,end = find_xyz(val, landmark)
        angle = calculate_angle2D(first,mid,end, 1) #각도 계산
        mid = mid[0:2]
        cv2.putText(frame, str(int(angle)),tuple(np.multiply(mid, [width,height]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0),5,cv2.LINE_AA)
        temp[key].append(angle)


      ####################################################################################################
      elbow_angle = temp['right_elbow'][-1]
      Is_elbow_up = (elbow_angle > 130)
      Is_elbow_down = (elbow_angle < 110)
      hip_angle = temp['right_hip'][-1]
      hip_condition = (hip_angle >140) and (hip_angle < 220)
      knee_angle = temp['right_knee'][-1]
      knee_condition = (knee_angle > 130) and (knee_angle < 205)
      lower_body_condition = hip_condition and knee_condition

      # 팔굽혀펴기 시작 자세 판단
      if not Is_start:
        Is_pushup_angle = (elbow_angle > 140) and (elbow_angle < 190) and (hip_angle >140) and (hip_angle < 200) and (knee_angle > 125) and (knee_angle < 190) 
        if Is_pushup_angle:
          ##########################################################################################
          # 여기에 음성으로 '시작합니다!' 라고 말해주면 좋을거같습니다. 이때부터 자세 평가 시작합니다.
          ##########################################################################################
          Is_start = True

      if not Is_start:
        temp['right_elbow'].pop()
        temp['right_hip'].pop()
        temp['right_knee'].pop()
      else:
        # identify if capctured image is outlier
        if IsOutlier_pushups(temp['right_elbow'], 0) or IsOutlier_pushups(temp['right_hip'], 1) or IsOutlier_pushups(temp['right_knee'], 2):
          temp['right_elbow'].pop()
          temp['right_hip'].pop()
          temp['right_knee'].pop()
        else:
      ####################################################################################################
          if Is_elbow_up and elbow_state == 'down' and lower_body_condition:

            # down state부터 up state까지 시간이 측정됨. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
            end = time.time()
            elbow_state = 'up'
            element = []

            # 팔꿈치를 핀 정도가 160도보다 큰 경우
            if max(temp['right_elbow']) > 160:
              element.append(1)
            # 팔꿈치를 덜 핀 경우
            else:
              element.append(0)

            # 팔꿈치를 굽힌 정도가 70도보다 작은 경우
            if min(temp['right_elbow']) < 70:
              element.append(1)
            # 팔꿈치를 덜 굽힌 경우
            else:
              element.append(0)
            # 팔꿈치 각도 데이터 초기화 (어플에서의 변수 저장 값이 많아지는 것을 방지하기 위해 팔굽 하나 할때마다 각도 축적해놓은거를 초기화함.)
            temp['right_elbow'] = []

            # 팔꿈치 굽혔다 필 동안 골반이 152도보다 내려갔는지(골반이 너무 아래로 내려갔는지)
            if min(temp['right_hip']) < 152:
              element.append(1)
            # 팔꿈치 굽혔다 필 동안 골반이 250도보다 올라갔는지(골반이 너무 위로 올라갔는지)
            elif max(temp['right_hip']) > 250:
              element.append(2)
            # 팔굽하는 동안 골반이 정상 자세(152~250도 사이값)인 경우
            else:
              element.append(0)
            temp['right_hip'] = []

            # knee condition
            if min(temp['right_knee']) < 130:
              element.append(0)
            else:
              element.append(1)
            temp['right_knee'] = []

            #Is_speed_good
            # down state부터 up state까지 시간차이가 1초가 되는지 판단함. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
            if end - start < 1:
              element.append(0)
            else:
              element.append(1)


            """
            개수 카운팅 되는 부분입니다. 이 부분에 음성으로 하나, 둘, 셋, ... 이렇게 개수를 세어주면 좋을거 같습니다.
            또한, element의 값을 보고 실시간으로 하는 피드백도 이 부분에 대입하는게 좋을거 같습니다.
            element는 [Is_elbow_up, Is_elbow_down, hip_condition, knee_condition, Is_speed_good]입니다. 각각에 대한 설명은 위 주석에 있습니다.
            """
            pushups.append(element)

          elif Is_elbow_down and elbow_state == 'up' and lower_body_condition:
            elbow_state = 'down'
            # down state부터 up state까지 시간이 측정됨. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
            start = time.time()

          cv2.putText(frame, str(len(pushups)), (200,300), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255),2, cv2.LINE_AA)

          out.write(frame)
  out.release()



In [ ]:
video_path = "/content/gdrive/MyDrive/국방부 해커톤/input/팔굽혀펴기/4.mp4"
cap = cv2.VideoCapture(video_path)
elbow_state = 'up'
Is_start = False

# 팔굽혀펴기 1회 측정 시간
start = 0
end = 0

#우측 관절값
joint_indx = {'right_elbow':[16,14,12],'right_hip':[12,24,26],'right_knee':[24,26,28]} 

"""
temp은 팔굽혀펴기 운동을 한번 할떄마다(내려갔다 올라올 때마다) 해당 관절 부분의 각도를 최신화함. 이전에 했던 팔굽혀펴기의 각도값들은 다 초기화함.
      -> 앱 상에서 관절 각도값을 저장하는 데에 무리가 가지않게 list를 초기화시켜줌.
"""
temp = {'right_elbow':[],'right_hip':[],'right_knee':[]}
pushups = []

"""
pushups는 list(list)형식이다. 안쪽 list는 length가 5이며 각각 Is_elbow_up, Is_elbow_down, hip_condition, knee_condition, Is_speed_good을 의미한다.
Is_elbow_up -> int 0 or 1 -> 팔꿈치를 완전히 펴면 1, 완전히 펴지 않으면 0
Is_elbow_down -> int 0 or 1 -> 팔꿈치를 완전히 굽히면 1, 완전히 굽히지 않으면 0
hip_condition -> int 0 or 1 or 2 -> 골반이 정상이면 0, 너무 내려가면 1, 너무 올라가면 2
knee_condition -> int 1 or 0 -> 무릎 각도가 정상범위면 1, 너무 내려가면 0
Is_speed_good -> int 1 or 0 -> 팔굽혀펴기 속도가 적당하면(1회당 1초 이상) 1, 너무 빠르면(1회당 1초 미만) 0 
예를 들어, pushup[0]이 [1,0,1,1,1]이면 첫번째 푸쉬업 동작이 올라올 때 팔꿈치를 완전히 폈고, 내려갈때 완전히 팔을 굽히지는 않았으며, 골반은 과도하게 내려갔고,
무릎각도는 정상범위이며 속도가 너무 빠르지않다는 의미이다.
"""

if cap.isOpened():
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    res=(int(width), int(height))
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
    out = cv2.VideoWriter('4test.mp4', fourcc, 20.0, res)

    frame = None
    while True:
      try:
        ret, frame = cap.read()
      except cv2.error:
        continue
      if not ret:
        break

      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      results = pose.process(image)

      if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        landmark = results.pose_landmarks.landmark
        for key,val in joint_indx.items():
          first,mid,end = find_xyz(val, landmark)
          angle = calculate_angle2D(first,mid,end, 1) #각도 계산
          mid = mid[0:2]
          cv2.putText(frame, str(int(angle)),tuple(np.multiply(mid, [width,height]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0),5,cv2.LINE_AA)
          temp[key].append(angle)


        ####################################################################################################
        elbow_angle = temp['right_elbow'][-1]
        Is_elbow_up = (elbow_angle > 130)
        Is_elbow_down = (elbow_angle < 110)
        hip_angle = temp['right_hip'][-1]
        hip_condition = (hip_angle >140) and (hip_angle < 220)
        knee_angle = temp['right_knee'][-1]
        knee_condition = (knee_angle > 130) and (knee_angle < 205)
        lower_body_condition = hip_condition and knee_condition

        # 팔굽혀펴기 시작 자세 판단
        if not Is_start:
          Is_pushup_angle = (elbow_angle > 140) and (elbow_angle < 190) and (hip_angle >140) and (hip_angle < 200) and (knee_angle > 125) and (knee_angle < 190) 
          if Is_pushup_angle:
            ##########################################################################################
            # 여기에 음성으로 '시작합니다!' 라고 말해주면 좋을거같습니다. 이때부터 자세 평가 시작합니다.
            ##########################################################################################
            Is_start = True

        if not Is_start:
          temp['right_elbow'].pop()
          temp['right_hip'].pop()
          temp['right_knee'].pop()
          continue

        # identify if capctured image is outlier
        if IsOutlier_pushups(temp['right_elbow'], 0) or IsOutlier_pushups(temp['right_hip'], 1) or IsOutlier_pushups(temp['right_knee'], 2):
            temp['right_elbow'].pop()
            temp['right_hip'].pop()
            temp['right_knee'].pop()
            continue   
        ##########################################################################################################
        
        if Is_elbow_up and elbow_state == 'down' and lower_body_condition:

          # down state부터 up state까지 시간이 측정됨. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
          end = time.time()

          elbow_state = 'up'
          element = []

          # 팔꿈치를 핀 정도가 160도보다 큰 경우
          if max(temp['right_elbow']) > 160:
            element.append(1)
          # 팔꿈치를 덜 핀 경우
          else:
            element.append(0)

          # 팔꿈치를 굽힌 정도가 70도보다 작은 경우
          if min(temp['right_elbow']) < 70:
            element.append(1)
          # 팔꿈치를 덜 굽힌 경우
          else:
            element.append(0)
          # 팔꿈치 각도 데이터 초기화 (어플에서의 변수 저장 값이 많아지는 것을 방지하기 위해 팔굽 하나 할때마다 각도 축적해놓은거를 초기화함.)
          temp['right_elbow'] = []

          # 팔꿈치 굽혔다 필 동안 골반이 152도보다 내려갔는지(골반이 너무 아래로 내려갔는지)
          if min(temp['right_hip']) < 152:
            element.append(1)
          # 팔꿈치 굽혔다 필 동안 골반이 250도보다 올라갔는지(골반이 너무 위로 올라갔는지)
          elif max(temp['right_hip']) > 250:
            element.append(2)
          # 팔굽하는 동안 골반이 정상 자세(152~250도 사이값)인 경우
          else:
            element.append(0)
          temp['right_hip'] = []

          # knee condition
          if min(temp['right_knee']) < 130:
            element.append(0)
          else:
            element.append(1)
          temp['right_knee'] = []

          #Is_speed_good
          # down state부터 up state까지 시간차이가 1초가 되는지 판단함. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
          if end - start < 1:
            element.append(0)
          else:
            element.append(1)


          """
          개수 카운팅 되는 부분입니다. 이 부분에 음성으로 하나, 둘, 셋, ... 이렇게 개수를 세어주면 좋을거 같습니다.
          또한, element의 값을 보고 실시간으로 하는 피드백도 이 부분에 대입하는게 좋을거 같습니다.
          element는 [Is_elbow_up, Is_elbow_down, hip_condition, knee_condition, Is_speed_good]입니다. 각각에 대한 설명은 위 주석에 있습니다.
          """
          pushups.append(element)

        elif Is_elbow_down and elbow_state == 'up' and lower_body_condition:
          elbow_state = 'down'
          # down state부터 up state까지 시간이 측정됨. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
          start = time.time()

        cv2.putText(frame, str(len(pushups)), (200,300), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255),2, cv2.LINE_AA)

      out.write(frame)
    out.release()



##3D angle

In [ ]:
video_path = "/content/wrong/7.mp4"
cap = cv2.VideoCapture(video_path)
elbow_state = 'up'

# 팔굽혀펴기 1회 측정 시간
start = 0
end = 0

#우측 관절값
joint_indx = {'right_elbow':[16,14,12],'right_hip':[12,24,26],'right_knee':[24,26,28]} 

"""
temp은 팔굽혀펴기 운동을 한번 할떄마다(내려갔다 올라올 때마다) 해당 관절 부분의 각도를 최신화함. 이전에 했던 팔굽혀펴기의 각도값들은 다 초기화함.
      -> 앱 상에서 관절 각도값을 저장하는 데에 무리가 가지않게 list를 초기화시켜줌.
"""
temp = {'right_elbow':[],'right_hip':[],'right_knee':[]}
pushups = []

"""
pushups는 list(list)형식이다. 안쪽 list는 length가 5이며 각각 Is_elbow_up, Is_elbow_down, hip_condition, knee_condition, Is_speed_good을 의미한다.
Is_elbow_up -> int 0 or 1 -> 팔꿈치를 완전히 펴면 1, 완전히 펴지 않으면 0
Is_elbow_down -> int 0 or 1 -> 팔꿈치를 완전히 굽히면 1, 완전히 굽히지 않으면 0
hip_condition -> int 0 or 1 or 2 -> 골반이 정상이면 0, 너무 내려가면 1, 너무 올라가면 2
knee_condition -> int 1 or 0 -> 무릎 각도가 정상범위면 1, 너무 내려가면 0
Is_speed_good -> int 1 or 0 -> 팔굽혀펴기 속도가 적당하면(1회당 1초 이상) 1, 너무 빠르면(1회당 1초 미만) 0 
예를 들어, pushup[0]이 [1,0,1,1,1]이면 첫번째 푸쉬업 동작이 올라올 때 팔꿈치를 완전히 폈고, 내려갈때 완전히 팔을 굽히지는 않았으며, 골반은 과도하게 내려갔고,
무릎각도는 정상범위이며 속도가 너무 빠르지않다는 의미이다.
"""

if cap.isOpened():
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    res=(int(width), int(height))
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
    out = cv2.VideoWriter('wtemp7.mp4', fourcc, 20.0, res)

    frame = None
    while True:
      try:
        ret, frame = cap.read()
      except cv2.error:
        continue
      if not ret:
        break

      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      results = pose.process(image)

      if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        for id, im in enumerate(results.pose_landmarks.landmark):
          h, w, c = frame.shape
          cx, cy = int(im.x*w), int(im.y*h)
          cv2.circle(frame, (cx,cy), 5, (255, 0, 0), cv2.FILLED)
        landmark = results.pose_landmarks.landmark
        for key,val in joint_indx.items():
          first,mid,end = find_xyz(val, landmark)
          angle = calculate_angle3D(first,mid,end, 1) #각도 계산
          mid = mid[0:2]
          cv2.putText(frame, str(angle),tuple(np.multiply(mid, [2400,1080]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,0),5,cv2.LINE_AA)
          temp[key].append(angle)

        elbow_angle = temp['right_elbow'][-1]
        Is_elbow_up = (elbow_angle > 137.5)
        Is_elbow_down = (elbow_angle < 127.5)

        
        hip_angle = temp['right_hip'][-1]
        hip_condition = (hip_angle >150) and (hip_angle < 220)
        knee_angle = temp['right_knee'][-1]
        knee_condition = (knee_angle > 152) and (knee_angle < 200)
        lower_body_condition = hip_condition and knee_condition

        if Is_elbow_up and elbow_state == 'down' and lower_body_condition:

          # down state부터 up state까지 시간이 측정됨. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
          end = time.time()

          elbow_state = 'up'
          element = []

          # 팔꿈치를 핀 정도가 160도보다 큰 경우
          if max(temp['right_elbow']) > 160:
            element.append(1)
          # 팔꿈치를 덜 핀 경우
          else:
            element.append(0)

          # 팔꿈치를 굽힌 정도가 90도보다 작은 경우
          if min(temp['right_elbow']) < 90:
            element.append(1)
          # 팔꿈치를 덜 굽힌 경우
          else:
            element.append(0)
          # 팔꿈치 각도 데이터 초기화 (어플에서의 변수 저장 값이 많아지는 것을 방지하기 위해 팔굽 하나 할때마다 각도 축적해놓은거를 초기화함.)
          temp['right_elbow'] = []

          # 팔꿈치 굽혔다 필 동안 골반이 160도보다 내려갔는지(골반이 너무 아래로 내려갔는지)
          if min(temp['right_hip']) < 160:
            element.append(1)
          # 팔꿈치 굽혔다 필 동안 골반이 220도보다 올라갔는지(골반이 너무 위로 올라갔는지)
          elif max(temp['right_hip']) > 220:
            element.append(2)
          # 팔굽하는 동안 골반이 정상 자세(160~220도 사이값)인 경우
          else:
            element.append(0)
          temp['right_hip'] = []

          # knee condition
          if min(temp['right_knee']) < 152:
            element.append(0)
          else:
            element.append(1)
          temp['right_knee'] = []

          #Is_speed_good
          # down state부터 up state까지 시간차이가 1초가 되는지 판단함. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
          if end - start < 1:
            element.append(0)
          else:
            element.append(1)


          """
          개수 카운팅 되는 부분입니다. 이 부분에 음성으로 하나, 둘, 셋, ... 이렇게 개수를 세어주면 좋을거 같습니다.
          또한, element의 값을 보고 실시간으로 하는 피드백도 이 부분에 대입하는게 좋을거 같습니다.
          element는 [Is_elbow_up, Is_elbow_down, hip_condition, knee_condition, Is_speed_good]입니다. 각각에 대한 설명은 위 주석에 있습니다.
          """
          pushups.append(element)

        elif Is_elbow_down and elbow_state == 'up' and lower_body_condition:
          elbow_state = 'down'
          # down state부터 up state까지 시간이 측정됨. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
          start = time.time()

        cv2.putText(frame, str(len(pushups)), (200,300), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255),2, cv2.LINE_AA)

      out.write(frame)
    out.release()



In [ ]:
pushups

[]

In [ ]:
len(pushups)

16

#**Evaulate Pushups**

In [ ]:
def pushups_to_score(pushups):
  each_score = []
  for e in pushups:
    Is_elbow_up = e[0]
    Is_elbow_down = e[1]
    Is_hip_good = 0
    if e[2] == 0:
      Is_hip_good = 1
    Is_knee_good = e[3]
    Is_speed_good = e[4]
    each_score.append(Is_elbow_up*25+Is_elbow_down*30+Is_hip_good*30+Is_knee_good*8+Is_speed_good*7)
  return each_score

In [ ]:
# pushups score for each reptition
pushups_to_score(pushups)

[62, 70, 40, 32, 32, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70]

In [ ]:
#total_score
sum(pushups_to_score(pushups))

1006

In [ ]:
from google.colab import files
files.download('wtemp6.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#**Checking Conditions for Squat**

In [ ]:
def get_distance(lm_from, lm_to):
  x2 = (lm_from.x - lm_to.x)**2
  y2 = (lm_from.y-lm_to.y)**2
  return (x2+y2)**0.5

In [ ]:
video_path = "/content/squat/4.mp4"
cap = cv2.VideoCapture(video_path)
squat_state = 'up'
Is_knee_out = False

# 스쿼트 1회 측정 시간
start = 0
end = 0


  # a = [215 for i in range(L)]
  # b = [240 for i in range(L)]
  # c = [147.5 for i in range(L)]

#우측 관절값
joint_indx = {'right_hip':[12,24,26],'right_knee':[24,26,28]} 

"""
temp_squat은 스쿼트 운동을 한번 할떄마다(내려갔다 올라올 때마다) 해당 관절 부분의 각도를 최신화함. 이전에 했던 스쿼트의 각도값들은 다 초기화함.
      -> 앱 상에서 관절 각도값을 저장하는 데에 무리가 가지않게 list를 초기화시켜줌.
"""
temp_squat = {'right_hip':[],'right_knee':[], 'average_hip_knee' : [], 'foot_length' : [], 'toe_location' : []} 
squats = []

"""
squats는 list(list)형식이다. 안쪽 list는 length가 5이며 각각 Is_relaxation, Is_contraction, Hip_knee_relation, Is_knee_in, Is_speed_good을 의미한다.
Is_relaxation -> int 0 or 1 -> hip과 무릎을 완전히 펴면 1, 완전히 펴지 않으면 0
Is_contraction -> int 0 or 1 -> hip과 무릎을 완전히 굽히면 1, 완전히 굽히지 않으면 0
Hip_knee_relation -> int 0 or 1 or 2 -> 스쿼트 시 엉덩이가 내려가면서 무릎을 굽히면(운동 시 엉덩이의 값과 무릎의 각도의 평균값이 180도와 비슷하면) 0, 무릎을 굽히지 않고 엉덩이를 뒤로 빼면 1,
                                        엉덩이를 뒤로 빼지 않고 무릎만 굽히면 2
Is_knee_in -> int 1 or 0 -> 무릎이 발끝(엄지발가락)을 넘지 않으면1, 넘어가면 0
Is_speed_good -> int 1 or 0 -> 스쿼트 속도가 적당하면(1회당 1초 이상) 1, 너무 빠르면(1회당 1초 미만) 0 
예를 들어, squats[0]이 [1,0,2,1,1]이면 첫번째 스쿼트 동작이 올라올 때 완전히 이완을 했고, 내려갈때 완전히 수축하진 않았으며(엉덩이를 더 내려야함),
엉덩이를 빼기보단 무릎만 굽혔고, 무릎이 발끝을 넘어가지 않았으며, 속도가 적당하다는 의미이다.
"""

if cap.isOpened():
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

  res=(int(width), int(height))
  fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
  out = cv2.VideoWriter('4out.mp4', fourcc, 20.0, res)

  frame = None
  while True:
    try:
      ret, frame = cap.read()
    except cv2.error:
      continue
    if not ret:
      break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    if results.pose_landmarks:
      mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
      landmark = results.pose_landmarks.landmark
      for key,val in joint_indx.items():
        first,mid,end = find_xyz(val, landmark)
        angle = calculate_angle3D(first,mid,end, 1) #각도 계산
        mid = mid[0:2]
        cv2.putText(frame, str(angle),tuple(np.multiply(mid, [width,height]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,0),5,cv2.LINE_AA)
        temp_squat[key].append(angle)
      knee_x = landmark[26].x # knee
      toe_x = landmark[32].x # toe

      ##########################수정사항############################
      if squat_state == 'up':
        temp_squat['foot_length'].append(get_distance(landmark[32], landmark[30]))
        temp_squat['toe_location'].append(toe_x)
      elif sum(temp_squat['toe_location'])/len(temp_squat['toe_location']) + sum(temp_squat['foot_length'])/len(temp_squat['foot_length'])*0.20 < knee_x:
        # 무릎이 발끝보다 조금 나간거는 허용해줌(스쿼트 특성상 정확히 무릎이 발끝보다 덜 나가지 않음. 또한 오차값도 고려해서 발길이의 20%는 threshold값으로 설정함)
        Is_knee_out = True
      ###############################################################

      hip_angle = temp_squat['right_hip'][-1]
      knee_angle = temp_squat['right_knee'][-1]
      temp_squat['average_hip_knee'].append((hip_angle + knee_angle) / 2)

      ###########수정############################
      Is_hip_up = hip_angle < 215
      Is_hip_down = hip_angle > 250
      Is_knee_up = knee_angle > 147.5
      ###########################################

      
      if Is_hip_up and Is_knee_up and squat_state == 'down':
        # down state부터 up state까지 시간이 측정됨. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
        end = time.time()

        squat_state = 'up'
        element = []
        # Is_relaxation
        # 엉덩이를 핀 정도가 195도보다 작은 경우(완전히 이완)
        if min(temp_squat['right_hip']) < 195:
          element.append(1)
        # 엉덩이를 덜 핀 경우
        else:
          element.append(0)

        # Is_contraction
        # 엉덩이를 굽힌 정도가 270도보다 큰 경우(완전히 수축)
        if max(temp_squat['right_hip']) > 270:
          element.append(1)
        # 엉덩이를 덜 굽힌 경우
        else:
          element.append(0)

        # 엉덩이 각도 데이터 초기화 (어플에서의 변수 저장 값이 많아지는 것을 방지하기 위해 스쿼트 하나 할때마다 각도 축적해놓은거를 초기화함.)
        temp_squat['right_hip'] = []
        temp_squat['right_knee'] = []


        # Hip_knee_relation
        # 스쿼트 동작동안 엉덩이와 무릎이 균형있게 내려가지 않고 엉덩이가 우선적으로 내려갔는지
        if max(temp_squat['average_hip_knee']) > 205:   #########수정
          element.append(1)
        # 스쿼트 동작동안 엉덩이와 무릎이 균형있게 내려가지 않고 무릎이 우선적으로 내려갔는지
        elif min(temp_squat['average_hip_knee']) < 165:   #########수정
          element.append(2)
        # 스쿼트 동작동안 엉덩이와 무릎이 균형있게 내려간 경우 (둘의 각도 평균값이 176~193의 경우)
        else:
          element.append(0)
        temp_squat['average_hip_knee'] = []

        # Is_knee_in
        # 무릎이 발끝보다 안쪽에 있는지 유무
        if Is_knee_out:
          element.append(0)
        else:
          element.append(1)
          
          #Is_speed_good
          # down state부터 up state까지 시간차이가 1초가 되는지 판단함. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
        if end - start < 1:
          element.append(0)
        else:
          element.append(1)

        """
        개수 카운팅 되는 부분입니다. 이 부분에 음성으로 하나, 둘, 셋, ... 이렇게 개수를 세어주면 좋을거 같습니다.
        또한, element의 값을 보고 실시간으로 하는 피드백도 이 부분에 대입하는게 좋을거 같습니다.
        element는 [Is_relaxation, Is_contraction, Hip_knee_relation, Is_knee_in, Is_speed_good]입니다. 각각에 대한 설명은 위 주석에 있습니다.
        """

        ######################추가사항###########################################
        temp_squat['foot_length'] = []
        temp_squat['toe_location'] = []
        #########################################################################
        squats.append(element)
        Is_knee_out = False

      elif Is_hip_down and (not Is_knee_up) and squat_state == 'up':
        squat_state = 'down'
        # down state부터 up state까지 시간이 측정됨. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
        start = time.time()

      cv2.putText(frame, str(len(squats)), (200,300), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255),2, cv2.LINE_AA)

    out.write(frame)
  out.release()



In [ ]:
squats

[[1, 1, 0, 1, 1], [1, 0, 0, 1, 1], [1, 1, 0, 1, 1], [1, 1, 2, 1, 1]]

#**Evaulate Squats**

In [ ]:
def squats_to_score(squats):
  each_score = []
  for e in squats:
    Is_relaxation = e[0]
    Is_contraction = e[1]
    Is_hip_knee_good = 0
    if e[2] == 0:
      Is_hip_knee_good = 1
    Is_knee_in = e[3]
    Is_speed_good = e[4]
    each_score.append(Is_relaxation*10+Is_contraction*20+Is_hip_knee_good*50+Is_knee_in*13+Is_speed_good*7)
  return each_score

In [ ]:
# squats score for each reptition
squats_to_score(squats)

[100, 80, 100, 50]

In [ ]:
sum(squats_to_score(squats))

330

In [ ]:
from google.colab import files
files.download('4out.mp4')


#**Calculating angle(2D) for Pullups**

In [ ]:
def calculate_angle2D(a,b,c,direction):
  """
  calculate_angle2D is divided by left and right side because this function uses external product
  input : a,b,c -> landmarks with shape [x,y,z,visibility]
          direction -> int -1 or 1
                      -1 means Video(photo) for a person's left side and 1 means Video(photo) for a person's right side
  output : angle between vector ba and bc with range 0~360
  """
  # external product's z value
  external_z = (b[0]-a[0])*(b[1]-c[1]) - (b[1]-a[1])*(b[0]-c[0])

  a = np.array(a[:2]) #first
  b = np.array(b[:2]) #mid
  c = np.array(c[:2]) #end

  ba = b-a
  bc = b-c
  dot_result = np.dot(ba, bc)


  ba_size = np.linalg.norm(ba)
  bc_size = np.linalg.norm(bc)
  radi = np.arccos(dot_result / (ba_size*bc_size))
  angle = np.abs(radi*180.0/np.pi)

  if external_z * direction > 0:
    angle = 360 - angle

  return angle


In [ ]:
def calculate_angle2D_vector(v1, v2):
  """
  calculate_angle2D_vector uses only internal product
  input : two lists(vectors) with length 2
          [x1, y1]
  output : angle between vector v1 and v2 with range 0~180
  """

  dot_result = np.dot(v1, v2)

  v1_size = np.linalg.norm(v1)
  v2_size = np.linalg.norm(v2)
  radi = np.arccos(dot_result / (v1_size*v2_size))
  
  angle = np.abs(radi*180.0/np.pi)

  return angle


#**Checking Conditions for Pullups**

##**Counting & Evaluation without using Segmentation**

In [ ]:
video_path = "/content/pullupside/r1.mp4"
cap = cv2.VideoCapture(video_path)
pullup_state = 'down'
Is_totally_contraction = False
Was_totally_contraction = False
human_pixel_cnt = 0


# 풀업 1회 측정 시간
start = 0
end = 0


# segmentation threshold
th = 0.65

# 95, 110

#우측 관절값
joint_indx = {'right_elbow':[16,14,12],'right_shoulder':[14,12,24]} 

"""
temp_pullup은 풀업 운동을 한번 할때마다(내려갔다 올라올 때마다) 해당 관절 부분의 각도를 최신화함. 이전에 했던 풀업의 각도값들은 다 초기화함.
      -> 앱 상에서 관절 각도값을 저장하는 데에 무리가 가지않게 list를 초기화시켜줌.
"""
temp_pullup = {'right_elbow':[],'right_shoulder':[], 'elbow_normY' : []} 
pullups = []

"""
pullups는 list(list)형식이다. 안쪽 list는 length가 4이며 각각 Is_relaxation, Is_contraction, Is_elbow_stable, Is_speed_good을 의미한다.
Is_relaxation -> int 0 or 1 -> 입의 높이가 손목의 높이보다 높게 올라가면(완전 수축하면) 1, 그렇지 않으면 0
Is_contraction -> int 0 or 1 -> hip과 무릎을 완전히 굽히면 1, 완전히 굽히지 않으면 0
Is_elbow_stable -> int 1 or 0 -> 팔꿈치와 지표면이 이루는 각도가 90도와 비슷하게 안정적이면 1, 각도의 변화량이 크면 0
Is_speed_good -> int 1 or 0 -> 풀업 속도가 적당하면(1회당 1.5초 이상) 1, 너무 빠르면(1회당 3초 미만) 0 
예를 들어, pullups[0]이 [1,0,1,1,1]이면 첫번째 풀업 동작이 올라올 때 완전히 이완을 했고, 내려갈때 완전히 수축하진 않았으며(입의 높이가 손목보다 높아야함),
완전 수축 시 숄더패킹이 되어있었으며, 팔꿈치는 안정적이었고, 속도가 적당하다는 의미이다.
"""


if cap.isOpened():
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

  res=(int(width), int(height))
  fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
  out = cv2.VideoWriter('r1out.mp4', fourcc, 20.0, res)

  frame = None
  output_image = None
  while True:
    try:
      ret, frame = cap.read()
    except cv2.error:
      continue
    if not ret:
      break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    if results.pose_landmarks:
      mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
      h, w, c = frame.shape
      landmark = results.pose_landmarks.landmark
      for key,val in joint_indx.items():
        first,mid,end = find_xyz(val, landmark)
        angle = calculate_angle2D(first,mid,end, 1) #각도 계산
        mid = mid[0:2]
        if key == 'right_shoulder' and angle < 30:
          angle = 359
        cv2.putText(frame, str(int(angle)),tuple(np.multiply(mid, [width,height]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,0),5,cv2.LINE_AA)
        temp_pullup[key].append(angle)

      # 손목-팔꿈치 직선과 y축 벡터가 이루는 각도를 계산 -> Is_elbow_stable에서 판단
      arm = [landmark[14].x-landmark[16].x, landmark[14].y-landmark[16].y]
      normY = [0, 1]
      temp_pullup['elbow_normY'].append(calculate_angle2D_vector(arm, normY))


      elbow_angle = temp_pullup['right_elbow'][-1]
      shoulder_angle = temp_pullup['right_shoulder'][-1]
      Is_elbow_up = (elbow_angle < 97.5)
      Is_elbow_down = (elbow_angle > 110) and (elbow_angle < 180)
      Is_shoulder_up = (shoulder_angle > 268 and shoulder_angle < 360)

      #  right mouth, elbow, wrist의 높이 측정
      # right_mouth의 높이
      right_mouth_y = landmark[10].y
      right_elbow_y = landmark[14].y
      right_wrist_y = landmark[16].y

      Is_mouth_upper_than_elbow = (right_mouth_y < right_elbow_y)
      Is_mouth_upper_than_wrist = (right_mouth_y < right_wrist_y)
      


      #################Is_totally_contraction(완전수축)의 정의####################
      if not Is_totally_contraction and Is_mouth_upper_than_wrist and elbow_angle < 100 and shoulder_angle > 280:
        Is_totally_contraction = True
      elif elbow_angle > 76 and (not Is_mouth_upper_than_wrist): 
        Is_totally_contraction = False
        Was_totally_contraction = True
      #############################################################################
              
      
      if Is_elbow_down and (not Is_shoulder_up) and pullup_state == 'up' and (not Is_mouth_upper_than_elbow):
        # down state부터 up state까지 시간이 측정됨. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
        end = time.time()

        pullup_state = 'down'
        element = []
        # Is_relaxation
        # 팔꿈치를 핀 정도가 145도보다 크고, 어깨를 핀 정도가 250도 보다 큰 경우(완전히 이완)
        if max(temp_pullup['right_elbow']) > 145 and min(temp_pullup['right_shoulder']) < 250:
          element.append(1)
        # 팔꿈치와 어깨를 덜 핀 경우
        else:
          element.append(0)

        # Is_contraction
        if Was_totally_contraction:
          element.append(1)
        else:
          element.append(0)
        Was_totally_contraction = False
        

        # 팔꿈치, 어깨 각도 데이터 초기화 (어플에서의 변수 저장 값이 많아지는 것을 방지하기 위해 풀업 하나 할때마다 각도 축적해놓은거를 초기화함.)
        temp_pullup['right_hip'] = []
        temp_pullup['right_knee'] = []
        Is_totally_contraction = False

        #Is_elbow_stable
        # 손목-팔꿈치 직선이 y축과 이루는 각도의 최대값이 25도 이하이면 운동 중 팔꿈치를 고정시킨 것으로 판단
        if max(temp_pullup['elbow_normY']) < 25:
          element.append(1)
        else:
          element.append(0)
        temp_pullup['elbow_normY'] = []
        
        #Is_speed_good
        # up state부터 down state까지 시간차이가 1.5초가 되는지 판단함. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
        if end - start < 1.5:
          element.append(0)
        else:
          element.append(1)

        """
        개수 카운팅 되는 부분입니다. 이 부분에 음성으로 하나, 둘, 셋, ... 이렇게 개수를 세어주면 좋을거 같습니다.
        또한, element의 값을 보고 실시간으로 하는 피드백도 이 부분에 대입하는게 좋을거 같습니다.
        element는 [Is_relaxation, Is_contraction, Hip_knee_relation, Is_knee_in, Is_speed_good]입니다. 각각에 대한 설명은 위 주석에 있습니다.
        """
        pullups.append(element)

      elif Is_elbow_up and Is_shoulder_up and pullup_state == 'down' and Is_mouth_upper_than_elbow:
        pullup_state = 'up'
        # up state부터 down state까지 시간이 측정됨. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
        start = time.time()
        

      
      cv2.putText(frame, str(len(pullups)), (200,300), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255),2, cv2.LINE_AA)
    out.write(frame)
  out.release()

In [ ]:
pullups

[[1, 1, 1, 0], [1, 1, 0, 0]]

##**Evaluate Pullups without shoulder packing**

In [ ]:
def pullups_to_score_without_seg(pullups):
  each_score = []
  for e in pullups:
    Is_relaxation = e[0]
    Is_contraction = e[1]
    Is_elbow_stable = e[2]
    Is_speed_good = e[3]
    each_score.append(Is_relaxation*15+Is_contraction*40+Is_elbow_stable*25+Is_speed_good*20)
  return each_score

In [ ]:
# pullups score for each reptition
pullups_to_score_without_seg(pullups)

[80, 55]

In [ ]:
sum(pullups_to_score_without_seg(pullups))

135

##**Counting & Evaluation with using Segmentation**

In [ ]:
import math
mp_selfie_segmentation = mp.solutions.selfie_segmentation
selfie_segmentation = mp_selfie_segmentation.SelfieSegmentation()


BG_COLOR = (192, 192, 192) # gray
MASK_COLOR = (255, 255, 255) # white

In [ ]:
video_path = "/content/pullupside/r1.mp4"
cap = cv2.VideoCapture(video_path)
pullup_state = 'down'
Is_totally_contraction = False
Was_totally_contraction = False
human_pixel_cnt = 0


# 풀업 1회 측정 시간
start = 0
end = 0


# segmentation threshold
th = 0.65

# 95, 110

#우측 관절값
joint_indx = {'right_elbow':[16,14,12],'right_shoulder':[14,12,24]} 

"""
temp_pullup은 풀업 운동을 한번 할때마다(내려갔다 올라올 때마다) 해당 관절 부분의 각도를 최신화함. 이전에 했던 풀업의 각도값들은 다 초기화함.
      -> 앱 상에서 관절 각도값을 저장하는 데에 무리가 가지않게 list를 초기화시켜줌.
"""
temp_pullup = {'right_elbow':[],'right_shoulder':[], 'elbow_normY' : [], 'human_pixel_ratio' : []} 
pullups = []

"""
pullups는 list(list)형식이다. 안쪽 list는 length가 5이며 각각 Is_relaxation, Is_contraction, Is_shoulder_packing, Is_elbow_stable, Is_speed_good을 의미한다.
Is_relaxation -> int 0 or 1 -> 입의 높이가 손목의 높이보다 높게 올라가면(완전 수축하면) 1, 그렇지 않으면 0
Is_contraction -> int 0 or 1 -> hip과 무릎을 완전히 굽히면 1, 완전히 굽히지 않으면 0
Is_shoulder_packing -> int 0 or 1 -> 완전 수축 시(입이 손목보다 높을 때) 숄더 패킹이 되어있는 상태면 1, 그렇지 않으면 0
Is_elbow_stable -> int 1 or 0 -> 팔꿈치와 지표면이 이루는 각도가 90도와 비슷하게 안정적이면 1, 각도의 변화량이 크면 0
Is_speed_good -> int 1 or 0 -> 풀업 속도가 적당하면(1회당 1.5초 이상) 1, 너무 빠르면(1회당 3초 미만) 0 
예를 들어, pullups[0]이 [1,0,1,1,1]이면 첫번째 풀업 동작이 올라올 때 완전히 이완을 했고, 내려갈때 완전히 수축하진 않았으며(입의 높이가 손목보다 높아야함),
완전 수축 시 숄더패킹이 되어있었으며, 팔꿈치는 안정적이었고, 속도가 적당하다는 의미이다.
"""


if cap.isOpened():
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

  res=(int(width), int(height))
  fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
  out = cv2.VideoWriter('r1out.mp4', fourcc, 20.0, res)

  frame = None
  output_image = None
  while True:
    try:
      ret, frame = cap.read()
    except cv2.error:
      continue
    if not ret:
      break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    if Is_totally_contraction: # 완전 수축된 경우에 segmentation 모델을 사용
      results_segmentation = selfie_segmentation.process(image)
      # Generate solid color images for showing the output selfie segmentation mask.
      fg_image = np.zeros(image.shape, dtype=np.uint8)
      fg_image[:] = MASK_COLOR
      bg_image = np.zeros(image.shape, dtype=np.uint8)
      bg_image[:] = BG_COLOR
      condition = np.stack((results_segmentation.segmentation_mask,) * 3, axis=-1) > th
      output_image = np.where(condition, fg_image, bg_image)

    if results.pose_landmarks:
      if not Is_totally_contraction:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
      else:
        mp_drawing.draw_landmarks(output_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
      h, w, c = frame.shape
      landmark = results.pose_landmarks.landmark
      for key,val in joint_indx.items():
        first,mid,end = find_xyz(val, landmark)
        angle = calculate_angle2D(first,mid,end, 1) #각도 계산
        mid = mid[0:2]
        if key == 'right_shoulder' and angle < 30:
          angle = 359
        if not Is_totally_contraction:
          cv2.putText(frame, str(int(angle)),tuple(np.multiply(mid, [width,height]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,0),5,cv2.LINE_AA)
        else:
          cv2.putText(output_image, str(int(angle)),tuple(np.multiply(mid, [width,height]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,0),5,cv2.LINE_AA)
        temp_pullup[key].append(angle)

      # 손목-팔꿈치 직선과 y축 벡터가 이루는 각도를 계산 -> Is_elbow_stable에서 판단
      arm = [landmark[14].x-landmark[16].x, landmark[14].y-landmark[16].y]
      normY = [0, 1]
      temp_pullup['elbow_normY'].append(calculate_angle2D_vector(arm, normY))


      elbow_angle = temp_pullup['right_elbow'][-1]
      shoulder_angle = temp_pullup['right_shoulder'][-1]
      Is_elbow_up = (elbow_angle < 97.5)
      Is_elbow_down = (elbow_angle > 110) and (elbow_angle < 180)
      Is_shoulder_up = (shoulder_angle > 268 and shoulder_angle < 360)

      #  right mouth, elbow, wrist의 높이 측정
      # right_mouth의 높이
      right_mouth_y = landmark[10].y
      right_elbow_y = landmark[14].y
      right_wrist_y = landmark[16].y

      Is_mouth_upper_than_elbow = (right_mouth_y < right_elbow_y)
      Is_mouth_upper_than_wrist = (right_mouth_y < right_wrist_y)
      

      
      ############## 완전 수축을 한 뒤로부터 segmentation을 시작##################
      ###################segmentation 부분########################################
      if Is_totally_contraction:
        # right_shoulder
        cx1, cy1 = int(landmark[12].x*w), int(landmark[12].y*h)
        # left_shoulder
        cx3 = int(landmark[11].x*w)
        cx1 = min(cx1, cx3)
        # right_hip
        cx2, cy2 = int(landmark[24].x*w), int(landmark[24].y*h)
        middle = (3*cy1+5*cy2) // 8
        length = abs(cy2 - middle)
        for y in range(middle, cy2):
          # print(output_image[y][cx1])
        # 해당 픽셀이 하얀색일 경우(사람 픽셀의 경우)
          if not ([255, 255, 255] - output_image[y][(3*cx1+cx2)//4]).any():
            human_pixel_cnt += 1
          # cv2.circle(output_image, ((3*cx1+cx2)//4,y), 2, (0, 255, 0), cv2.FILLED) -> 이부분 있으면 해당픽셀 색깔값이 잘 안잡힘
        temp_pullup['human_pixel_ratio'].append(human_pixel_cnt / length)
        human_pixel_cnt = 0
      ############################################################################
      ############################################################################



      #################Is_totally_contraction(완전수축)의 정의####################
      if not Is_totally_contraction and Is_mouth_upper_than_wrist and elbow_angle < 100 and shoulder_angle > 280: ## 이때부터 segmentation 시작
        Is_totally_contraction = True
      elif elbow_angle > 76 and (not Is_mouth_upper_than_wrist): ### 이때 segmentation 종료
        Is_totally_contraction = False
        Was_totally_contraction = True
      #############################################################################
              
      
      if Is_elbow_down and (not Is_shoulder_up) and pullup_state == 'up' and (not Is_mouth_upper_than_elbow):
        # down state부터 up state까지 시간이 측정됨. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
        end = time.time()

        pullup_state = 'down'
        element = []
        # Is_relaxation
        # 팔꿈치를 핀 정도가 145도보다 크고, 어깨를 핀 정도가 250도 보다 큰 경우(완전히 이완)
        if max(temp_pullup['right_elbow']) > 145 and min(temp_pullup['right_shoulder']) < 250:
          element.append(1)
        # 팔꿈치와 어깨를 덜 핀 경우
        else:
          element.append(0)

        # Is_contraction
        if Was_totally_contraction:
          element.append(1)
        else:
          element.append(0)
        Was_totally_contraction = False
        

        # 팔꿈치, 어깨 각도 데이터 초기화 (어플에서의 변수 저장 값이 많아지는 것을 방지하기 위해 풀업 하나 할때마다 각도 축적해놓은거를 초기화함.)
        temp_pullup['right_hip'] = []
        temp_pullup['right_knee'] = []
        Is_totally_contraction = False

        #Is_shoulder_packing
        ratio_sum = sum(temp_pullup['human_pixel_ratio'])
        length_ratio = len(temp_pullup['human_pixel_ratio'])
        Is_empty = (len(temp_pullup['human_pixel_ratio']) == 0)
        if Is_empty or ((ratio_sum / length_ratio) > 0.5):
          element.append(0)
        else: # 어깨-골반 높이의 중심과 골반의 높이까지 선을 이었을 때 절반 이하가 사람인 픽셀이면 숄더패킹을 한 것으로 간주
          element.append(1)
        temp_pullup['human_pixel_ratio'] = []

        #Is_elbow_stable
        # 손목-팔꿈치 직선이 y축과 이루는 각도의 최대값이 25도 이하이면 운동 중 팔꿈치를 고정시킨 것으로 판단
        if max(temp_pullup['elbow_normY']) < 25:
          element.append(1)
        else:
          element.append(0)
        temp_pullup['elbow_normY'] = []
        
        #Is_speed_good
        # up state부터 down state까지 시간차이가 1.5초가 되는지 판단함. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
        if end - start < 1.5:
          element.append(0)
        else:
          element.append(1)

        """
        개수 카운팅 되는 부분입니다. 이 부분에 음성으로 하나, 둘, 셋, ... 이렇게 개수를 세어주면 좋을거 같습니다.
        또한, element의 값을 보고 실시간으로 하는 피드백도 이 부분에 대입하는게 좋을거 같습니다.
        element는 [Is_relaxation, Is_contraction, Hip_knee_relation, Is_knee_in, Is_speed_good]입니다. 각각에 대한 설명은 위 주석에 있습니다.
        """
        pullups.append(element)

      elif Is_elbow_up and Is_shoulder_up and pullup_state == 'down' and Is_mouth_upper_than_elbow:
        pullup_state = 'up'
        # up state부터 down state까지 시간이 측정됨. 다만 코드 실행시 시간측정이라 real time에서도 잘 작동하는지 봐야함. 연산에 의한 latency가 발생할 수 있기 때문
        start = time.time()
        

      
      if not Is_totally_contraction:
        cv2.putText(frame, str(len(pullups)), (200,300), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255),2, cv2.LINE_AA)
      else:
        cv2.putText(output_image, str(len(pullups)), (200,300), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255),2, cv2.LINE_AA)

    if not Is_totally_contraction:
      out.write(frame)
    else:
      out.write(output_image)
  out.release()

In [ ]:
pullups

[[1, 1, 1, 1, 0], [1, 1, 1, 0, 0]]

In [ ]:
pullups

[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]

##**Evaulate Pullups with segmentation**

In [ ]:
def pullups_to_score(pullups):
  each_score = []
  for e in pullups:
    Is_relaxation = e[0]
    Is_contraction = e[1]
    Is_shoulder_packing = e[2]
    Is_elbow_stable = e[3]
    Is_speed_good = e[4]
    each_score.append(Is_relaxation*10+Is_contraction*20+Is_shoulder_packing*40+Is_elbow_stable*20+Is_speed_good*10)
  return each_score

In [ ]:
# pullups score for each reptition
pullups_to_score(pullups)

[90, 70]

In [ ]:
# total score
sum(pullups_to_score(pullups))

160